# Day 26 — "Change Detection Architectures & Losses (Remote Sensing Focus)"

Change detection compares two time points and segments the difference. Architectures and losses must handle extreme imbalance and temporal noise.


In [1]:
# Ensure repo root is on sys.path for local imports
import sys
from pathlib import Path

repo_root = Path.cwd()
if not (repo_root / "days").exists():
    for parent in Path.cwd().resolve().parents:
        if (parent / "days").exists():
            repo_root = parent
            break

sys.path.insert(0, str(repo_root))
print(f"Using repo root: {repo_root}")


Using repo root: /media/abdul-aziz/sdb7/masters_research/math_course_dlcv


## 1. Core Intuition

- Change detection segments **differences over time**, not objects.
- Illumination, seasonality, misalignment, and sensor noise make the task noisy.
- Architectures compare features using shared weights to ensure fair comparison.


## 2. What Makes Change Detection Unique

| Aspect | Segmentation | Change Detection |
| --- | --- | --- |
| Input | Single image | Image pair (T1, T2) |
| Target | Object class | Change / no-change |
| Noise | Moderate | High |
| Imbalance | Medium | Extreme |


## 3. Architecture Families

- **Siamese networks**: shared encoder for both images, then compare features.
- **Early fusion**: concatenate T1/T2 at input (simple but noisy).
- **Late fusion**: compare feature maps (difference, concat, attention).
- **Change UNet**: Siamese encoder + UNet decoder with multi-scale skips.


## 4. Feature Difference Operators

Common choices:

- absolute difference |f1 - f2|
- concatenation [f1, f2, |f1 - f2|]
- attention-guided fusion


## 5. Loss Functions for Change Detection

- BCE is dominated by "no-change" pixels.
- Dice and Focal focus on rare change regions.
- Best practice: BCE + Dice or Focal + Dice.


## 6. Python — Synthetic Change Pair + Losses

`days/day26/code/change_detection.py` creates synthetic T1/T2 images, a change mask, and reports BCE/Dice/Focal losses.


In [2]:
from days.day26.code.change_detection import (
    make_pair,
    binary_cross_entropy,
    dice_loss,
    focal_loss,
)

t1, t2, change = make_pair()
prob = (abs(t2 - t1) / (abs(t2 - t1).max() + 1e-6))

print("BCE:", binary_cross_entropy(prob, change))
print("Dice:", dice_loss(prob, change))
print("Focal:", focal_loss(prob, change))


BCE: 0.07897105813026428
Dice: 0.42738252878189087
Focal: 0.05634211748838425


## 7. Visualization — Change Detection Pipeline

`days/day26/code/visualizations.py` plots T1, T2, predicted change, and ground truth.


In [3]:
from days.day26.code.visualizations import plot_change_pipeline

RUN_FIGURES = False

if RUN_FIGURES:
    plot_change_pipeline()
else:
    print("Set RUN_FIGURES = True to regenerate Day 26 figures inside days/day26/outputs/.")


Set RUN_FIGURES = True to regenerate Day 26 figures inside days/day26/outputs/.


## 8. Metrics

Evaluate using IoU, F1, precision/recall. Accuracy is misleading due to imbalance.


## 9. Post-Processing

Connected components filtering removes tiny false positives and enforces connectivity.


## 10. Mini Exercises

1. Compare early vs late fusion on a small dataset.
2. Train with BCE only vs BCE + Dice and compare recall.
3. Visualize seasonal false positives.
4. Apply connected-component filtering to reduce noise.


## 11. Key Takeaways

- Change detection is comparative learning, not classification.
- Siamese encoders dominate in remote sensing.
- Feature differencing is the core operator.
- Dice/Focal losses handle extreme imbalance.
- Metrics must focus on the changed pixels.
